## Setup

For these examples, you'll need:

1. numpy
2. matplotlib
3. qiskit, version >=1.0 and <2.0
4. qiskit-ionq
5. An IonQ API key

If this notebook was not launched from an environment where Qiskit and Qiskit-IonQ are installed, uncomment and run the next cell to install them in this notebook's kernel.

In [ ]:
#%pip install "qiskit==1.4" qiskit-ionq numpy

In [ ]:
import numpy as np

Set your API key as an environment variable from here, if needed:

In [ ]:
import os
os.environ["IONQ_API_KEY"] = "YOUR API KEY HERE"

Set up the IonQProvider for Qiskit:

In [ ]:
from qiskit_ionq import IonQProvider
provider = IonQProvider()

# Transpilation and native gates

## A note on accepted QIS gates

Our API and compiler will directly accept [these gates](), as well as their controlled and multi-controlled variants:
* x	Pauli X gate
* y	Pauli Y gate
* z	Pauli Z gate
* rx	X-axis rotation
* ry	Y-axis rotation
* rz	Z-axis rotation
* h	Hadamard gate
* not	Convenient alias for Pauli-X gate
* cnot	Convenient alias for controlled-not gate
* s	S gate
* si	Conjugate transpose of S gate
* t	T gate
* ti	Conjugate transpose of T gate
* v	Square root of not gate
* vi	Conjugate transpose of square-root-of-not gate
* swap	Swaps two qubits

In many cases, other gates, operations, and aliases included in Qiskit (or another SDK) are automatically converted to these by qiskit-ionq (or the equivalent integration). However, in some cases you might need to use qiskit's `transpile` function to convert from special gates or higher-level operations to the set of QIS gates accepted by IonQ backends.

When you submit a circuit using these QIS gates, whatever you pass in will be further optimized by IonQ's transpiler, and converted to our native gateset. However, if you submit a circuit using IonQ's native gates directly, it will fully bypass our compiler - which means it won't be optimized or modified.

## Introducing the native gates

The GPi, GPi2, MS, and ZZ gates are included in `qiskit_ionq`.

**Note**: All of these gates accept angle parameters in _turns_, not _radians_, where 1 turn = $2\pi$ radians.

In [ ]:
from qiskit_ionq.ionq_gates import GPIGate, GPI2Gate, MSGate, ZZGate

You can look at the values of the gate matrices with any input parameter:

In [ ]:
GPIGate(0).to_matrix()

You can compare these to QIS gate matrices:

In [ ]:
from qiskit.circuit.library import RXGate, RYGate, RXXGate

In [ ]:
RXXGate(np.pi/2).to_matrix() * np.sqrt(2)

In [ ]:
MSGate(0, 0, 0.25).to_matrix() * np.sqrt(2)

In [ ]:
np.isclose(
    RXXGate(np.pi/2).to_matrix() * np.sqrt(2),
    MSGate(0, 0, 0.25).to_matrix() * np.sqrt(2)
)

The fully entangling MS gate with no phase offset is equivalent to RXX with an angle of $\frac{\pi}{2}$

## Constructing a circuit in native gates

You can build a circuit directly in native gates, either designing it in the native gates directly or applying gate decompositions and conversions manually (more information is available in our docs).

Let's build something like the "Hello world" Bell state example:

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2, name="Native gates example 1")

# Hadamard
qc.append(GPIGate(0), [0]) # Rx(pi)
qc.append(GPI2Gate(0.25), [0]) # Ry(pi/2)

# CNOT to XX: https://arxiv.org/abs/1603.07678
qc.append(GPI2Gate(0.25), [0]) # Ry(pi/2)
qc.append(MSGate(0,0,0.25), [0,1]) # XX(pi/4)
qc.append(GPI2Gate(0.5), [0]) # Rx(-pi/2)
qc.append(GPI2Gate(0.5), [1]) # Rx(-pi/2)
qc.append(GPI2Gate(-0.25), [0]) # Ry(pi/2)

qc.measure_all()

In [ ]:
qc.draw()

We can consolidate some gates here: two consecutive GPi2 gates with the same $\phi$ (two $\frac{\pi}{2}$ rotations around the same axis) are just a single GPi gate with that $\phi$ (one $\pi$ rotation around that axis).

When we submit in native gates, we fully bypass IonQ's compiler - so these gates wouldn't be combined automatically before execution.

In [ ]:
qc = QuantumCircuit(2, name="Native gates example 1")

qc.append(GPIGate(0), [0])
qc.append(GPIGate(0.25), [0]) # consolidated from two GPI2Gates
qc.append(MSGate(0,0,0.25), [0,1])
qc.append(GPI2Gate(0.5), [0])
qc.append(GPI2Gate(0.5), [1])
qc.append(GPI2Gate(-0.25), [0])

qc.measure_all()

In [ ]:
qc.draw()

To run a native gate circuit, we need a backend that is set to accept native gates. The simulator (including with noise models) and QPU can run native gate circuits when the backend is set up with the option `gateset="native"`. The default gateset is `qis`.

Note that MS gates are accepted by Aria systems and the simulator with Aria noise model, while ZZ gates are accepted by Forte systems and the simulator with Forte noise model.

In [ ]:
backend_sim_native = provider.get_backend("simulator", gateset="native")

Let's run this circuit with the ideal simulator:

In [ ]:
job1_ideal = backend_sim_native.run(qc)

In [ ]:
job1_ideal.get_probabilities()

Similarly we can run with a noise model (making sure the noise model matches the gates we're using - MS for Aria, ZZ for Forte)

In [ ]:
backend_sim_native_aria = provider.get_backend("simulator", gateset="native")
backend_sim_native_aria.set_options(noise_model="aria-1")

In [ ]:
job1_noisy = backend_sim_native_aria.run(qc, shots=1000)

In [ ]:
job1_noisy.get_counts()

And we can run on QPU (optional):

In [ ]:
backend_aria_native = provider.get_backend("qpu.aria-1", gateset="native")
job1_aria = backend_aria_native.run(qc, shots=100)

Cancel the job, or retrieve it later:

In [ ]:
job1_aria.cancel()

In [ ]:
#job1_aria = aria_native.retrieve_job(job1_aria.job_id())
#job1_aria.get_counts()

## Transpiling a circuit to native gates

We can also use Qiskit's transpiler to convert a circuit from standard QIS gates to native gates before submitting to an IonQ native-gate backend. This is only supported for Aria systems (MS gates) currently, but support for Forte systems (ZZ gates) will be coming soon.

First build another "Hello world" circuit using QIS gates:

In [ ]:
qc_qis = QuantumCircuit(2, name="QIS gate example")
qc_qis.h(0)
qc_qis.cx(0, 1)
qc_qis.measure_all()

Use Qiskit's transpiler with an IonQ native gate backend:

In [ ]:
backend_sim_native = provider.get_backend("simulator", gateset="native")

In [ ]:
from qiskit import transpile

In [ ]:
qc_native = transpile(qc_qis, backend=backend_sim_native)

This uses qiskit's optimization (which you can control via the `optimization_level` parameters) and then conversion to IonQ's native gates.

Submit the same way as before:

In [ ]:
job2_ideal = backend_sim_native.run(qc_native)
job2_ideal.get_probabilities()

With this approach, you have more direct control over the circuit that is being submitted and run (for example, if you wanted to submit a circuit containing gates that would be optimized out, you could set qiskit's optimization_level to zero when transpiling to IonQ's native gates), but you won't get the potential performance benefit from IonQ's compiler.

More compiler options and visibility will be available in the future.

More information about native gates, including different SDK examples, can be found in our docs [here](https://docs.ionq.com/guides/getting-started-with-native-gates).